In [1]:
from pathlib import Path
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pilot_academy.data.io import read_fn

In [ ]:
# Paths
PROJECT_DIR = Path("/workspaces/pilot_academy")
DATA_DIR = PROJECT_DIR / "data" 
OUT_DIR = PROJECT_DIR / "outputs" 
CONFIG_DIR = PROJECT_DIR / "config"

DATASET_NAME = "all_towns_with_weather"
DS_DIR = DATA_DIR / f"{DATASET_NAME}" / "raw"
IMAGES_DIR = DS_DIR / "images"

df_path = DS_DIR / "df_annotations.csv"
label_to_id_path = DS_DIR / "label_to_id.json"
action_to_id_path = DS_DIR / "action_to_id.json"

In [ ]:
# Load annotations
df_annotations = pd.read_csv(df_path, index_col="frame_index")

# Load mappings
with open(action_to_id_path) as f:
    action_to_id = json.load(f)

with open(label_to_id_path) as f:
    label_to_id = {
        tuple(k.split("|")): int(v)
        for k, v in json.load(f).items()
    }


# Collect images
image_paths = sorted(IMAGES_DIR.glob("*.png"))

# map filename -> full path
paths_by_name = {p.name: p for p in image_paths}

needed = set(df_annotations["image_name"])
available = set(paths_by_name.keys())

missing = needed - available
if missing:
    raise FileNotFoundError(
        f"{len(missing)} missing image(s). First: {next(iter(missing))}"
    )


# Configuration
crop_ymin = 100
grayscale = True

configured_read_fn = partial(
    read_fn,
    crop_ymin=crop_ymin,
    grayscale=grayscale,
)